[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/110XwsAvyjjGKVjsenR98r4AoFohbFSZH)


## Preparing Dataset

*NOTICE: fetching Dataset is not working colab (due to sanctions or what) so I provided the corpus alongside the notebook. You can either fetch dataset using colab via connecting it to a local machine or simply use the corpus.txt .*

### Method 1: Fetching Dataset

#### Importing Libraries

In [ ]:
import requests
from bs4 import BeautifulSoup

#### Setting URL

In [ ]:
URL = "https://ganjoor.net/moulavi/shams/ghazalsh/sh"
ghazalha = range(1501, 2001)
corpus = ""

##### Webscraping Data From Ganjoor

In [ ]:
for ghazal in ghazalha:
    page = requests.get(URL+str(ghazal))

    soup = BeautifulSoup(page.content, "html.parser")

    results = soup.find(id="garticle")

    beitha = results.find_all("div", class_="b")

    print("غزل شماره " + str(ghazal))
    for beit in beitha:
        mesra1 = beit.find("div", class_="m1")
        mesra2 = beit.find("div", class_="m2")
        print(mesra1.text)
        corpus += mesra1.text + " "
        print(mesra2.text)
        corpus += mesra2.text + " "
        print()
    print()

#### Saving Data in a Text File (optional)

In [ ]:
text_file = open("corpus.txt", "w", encoding="utf-8")
text_file.write(corpus)
text_file.close()

In [ ]:
corpus

### Method 2: Use pre-made dataset

In [ ]:
!curl -o corpus.txt https://raw.githubusercontent.com/AmirHajimohamadi/nlp/master/corpus.txt

with open('corpus.txt', 'r') as file:
    corpus = file.read().rstrip()
file.close()


!curl -o stopwords.txt https://raw.githubusercontent.com/AmirHajimohamadi/nlp/master/stopwords.txt

with open('stopwords.txt', 'r') as file:
    stopwords = file.read().rstrip()
file.close()


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  585k  100  585k    0     0  1106k      0 --:--:-- --:--:-- --:--:-- 1106k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6131  100  6131    0     0  16525      0 --:--:-- --:--:-- --:--:-- 16481


### Preprocessing the corpus

#### Importing Libraries

In [ ]:
import numpy as np
from tqdm import tqdm

In [ ]:
stopwords = stopwords.replace("\n", " ")
stopwords = stopwords.split()

In [ ]:
text = corpus.split()
text = [w.replace('\u200c', '') for w in text]

Removing stopwords from the text

In [ ]:
text = [w for w in text if not w in stopwords]

### Declare some variables

In [ ]:
# Defining the window for context
window = 2
# Creating a placeholder for the scanning of the word list
word_lists = []

getting context words and main words:

In [ ]:
for i, word in enumerate(text):
  for w in range(window):
    # Getting the context that is ahead by *window* words
    if i + 1 + w < len(text): 
      word_lists.append([word] + [text[(i + 1 + w)]])
    # Getting the context that is behind by *window* words
    if i - w - 1 >= 0:
      word_lists.append([word] + [text[(i - w - 1)]])

A dictionary of unique words in corpus:

In [ ]:
def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict    

### Cooking the model food

In [ ]:
unique_word_dict = create_unique_word_dict(text)

# Defining the number of features (unique words)
n_words = len(unique_word_dict)

# Getting all the unique words 
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

Making X and Y matrices

In [ ]:
for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words, dtype=np.int8)
    Y_row = np.zeros(n_words, dtype=np.int8)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)


332910it [00:07, 44941.77it/s]


In [ ]:
X = np.array(X, dtype=np.int8)
Y = np.array(Y, dtype=np.int8)